<a href="https://colab.research.google.com/github/hosein9574/My-agents/blob/main/Copy_of_RAG_chromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install transformers --upgrade
!pip install sentence-transformers --upgrade
!pip install chromadb --upgrade
!pip install gradio --upgrade

  Using cached gradio-5.29.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 104.9 MB/s eta 0:00:00


In [ ]:
# Imports
import numpy as np
import pandas as pd
import transformers
import sentence_transformers
from sentence_transformers import SentenceTransformer
import chromadb
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gradio as gr

In [ ]:
# Check versions
print("Transformers version:", transformers.__version__)
print("Sentence-Transformers version:", sentence_transformers.__version__)
print("ChromaDB version:", chromadb.__version__)

Transformers version: 4.51.3
Sentence-Transformers version: 4.1.0
ChromaDB version: 1.0.8


In [ ]:
# Set up Kaggle API (make sure kaggle.json is uploaded in Colab first)
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download and unzip BBC News dataset
!kaggle datasets download -d gpreda/bbc-news
!unzip -o bbc-news.zip

Dataset URL: https://www.kaggle.com/datasets/gpreda/bbc-news
License(s): CC0-1.0
Archive:  bbc-news.zip
  inflating: bbc_news.csv            


In [ ]:
# Load data
news = pd.read_csv('./bbc_news.csv')
MAX_NEWS = 1000
DOCUMENT = "description"
TOPIC = "title"
news["id"] = news.index
subset_news = news.head(MAX_NEWS)

In [ ]:
# Setup ChromaDB
chroma_client = chromadb.PersistentClient(path="./chromadb")
collection_name = "news_collection_" + datetime.now().strftime("%s")

In [ ]:
# Safe check for existing collection
existing_names = [col.name for col in chroma_client.list_collections()]
if collection_name in existing_names:
    chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)

In [ ]:
# Embed and add to ChromaDB
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.encode(subset_news[DOCUMENT].tolist(), convert_to_numpy=True)
collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
    embeddings=embeddings.tolist(),
)

In [ ]:
# Load language model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",  # Use GPU if available
)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# QA function
def answer_question(user_question):
    try:
        results = collection.query(query_texts=[user_question], n_results=5)
        context = "\n".join(results["documents"][0])
        context = context[:5120]  # Truncate if needed

        prompt = f"""
        Relevant context: {context}
        Considering the relevant context, answer the question.
        Question: {user_question}
        Answer: """

        response = pipe(prompt)
        answer = response[0]["generated_text"].split("Answer:")[-1].strip()
        return answer
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Gradio Chat Interface
def chat_function(message, history):
    return answer_question(message)

gr.ChatInterface(fn=chat_function, title="📰 News QA Bot", description="Ask anything about recent news articles.").launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7634b49b4dc56eeef8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
